In [1]:
import re
import numpy as np

In [2]:
def read_input(day):
    with open(f'Inputs/input{day}') as f:
        return f.read().rstrip('\n')
#Norvig functions
def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def Array(lines):
    "Parse an iterable of str lines into a 2-D array. If `lines` is a str, splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return mapt(Vector, lines)

def Vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(Atom, line.replace(',', ' ').split())

def Integers(text): 
    "Return a tuple of all integers in a string."
    return mapt(int, re.findall(r'-?\b\d+\b', text))

def Integers2(text): 
    "Return a tuple of all integers in a string."
    return mapt(int, re.findall(r'\d+', text))

def Atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

# Day 25

In [7]:
points = Array(read_input(25))
points[:5]

((-8, -4, -4, -5), (0, 8, 3, 8), (0, 0, 8, -2), (2, 6, 4, -2), (6, 5, 0, -7))

In [8]:
def manhattan_distance(a, b):
    if len(a) != len(b):
        raise ValueError('a and b must be the same length')
    return sum([abs(b[i] - a[i]) for i in range(len(a))])
    
assert(manhattan_distance((0,0), (3,4)) == 7)
assert(manhattan_distance((0,-1,0), (1,1,2)) == 5)

In [21]:
def in_group_range(point, group, max_dist=3):
    return any([manhattan_distance(point, x) <= max_dist for x in group])

In [41]:
def constellations(points, max_dist=3):
    groups = []
    for point in points:
        new_group = [point]
        group_copy = groups[:]
        for group in group_copy:
            if in_group_range(point, group, max_dist=max_dist):
                new_group.extend(group)
                groups.remove(group)
        groups.append(new_group)
    return groups

In [42]:
len(constellations(Array("""
-1,2,2,0
0,0,2,-2
0,0,0,-2
-1,2,0,0
-2,-2,-2,2
3,0,2,-1
-1,3,2,2
-1,0,-1,0
0,2,1,-2
3,0,0,0
""".strip())))

4

In [43]:
len(constellations(Array("""
1,-1,0,1
2,0,-1,0
3,2,-1,0
0,0,3,1
0,0,-1,-1
2,3,-2,0
-2,2,0,0
2,-2,0,-1
1,-1,0,-1
3,2,0,2
""".strip())))

3

In [44]:
len(constellations(Array("""
1,-1,-1,-2
-2,-2,0,1
0,2,1,3
-2,3,-2,1
0,2,3,-2
-1,-1,1,-2
0,-2,-1,0
-2,2,3,-1
1,2,2,0
-1,-2,0,-2
""".strip())))

8

In [45]:
len(constellations(points))

394

# Day 24

In [296]:
class Group():
    
    def __init__(self, army, num_units, unit_power, unit_hp, initiative, attack_type, weaknesses=None, immunities=None):
        self.army = army
        self.num_units = num_units
        self.unit_power = unit_power
        self.unit_hp = unit_hp
        self.initiative = initiative
        self.attack_type = attack_type
        self.weaknesses = weaknesses if weaknesses else []
        self.immunities = immunities if immunities else []  

    def receive_damage(self, amount):
        killed_units = amount // self.unit_hp
        self.num_units = max(0, self.num_units - killed_units)
        
    def is_enemy(self, army):
        return self.army != army
    
    @property
    def effective_power(self):
        return self.unit_power * self.num_units
    
    def __repr__(self):
        return f"{self.army} group with {self.num_units} units. Attack type: {self.attack_type}. Immune to {self.immunities}"

In [53]:
g = Group(army='immune', num_units=18, unit_power=8, unit_hp=729, initiative=10, attack_type='radiation', 
          weaknesses=['fire'], immunities=['cold', 'slashing'])
g.effective_power

144

In [98]:
def target_selection(all_groups):
    targets = {}
    for attacker in sorted(all_groups, key=attacker_priority, reverse=True):
        candidate_targets = [group for group in all_groups if attacker.is_enemy(group.army) and group not in targets.values()]
        if candidate_targets:
            best_target = max(candidate_targets, key=target_prioritisation(attacker))
            if maximal_damage(attacker, best_target) > 0:
                targets[attacker] = best_target
    return targets

In [73]:
def damage_phase(targets):
    for attacker, defender in sorted(targets.items(), key=lambda x: x[0].initiative, reverse=True):
        defender.receive_damage(maximal_damage(attacker, defender))

In [74]:
def fight(all_groups):
    targets = target_selection(all_groups)
    damage_phase(targets)
    return [g for g in all_groups if g.num_units > 0]

In [104]:
imm1 = Group(army='immune', num_units=17, unit_hp=5390, unit_power=4507, attack_type='fire', initiative=2, 
          weaknesses=['radiation', 'bludgeoning'])
imm2 = Group(army='immune', num_units=989, unit_hp=1274, unit_power=25, attack_type='slashing', initiative=3,  
          weaknesses=['slashing', 'bludgeoning'], immunities=['fire'])

inf1 = Group(army='infection', num_units=801, unit_hp=4706, unit_power=116, attack_type='bludgeoning', initiative=1, 
          weaknesses=['radiation'])
inf2 = Group(army='infection', num_units=4485, unit_hp=2961, unit_power=12, attack_type='slashing', initiative=4, 
          weaknesses=['fire', 'cold'], immunities=['radiation'])

all_groups = [imm1, imm2, inf1, inf2]

In [100]:
target_selection(all_groups)

{immune group with 17 units: infection group with 4485 units,
 immune group with 989 units: infection group with 801 units,
 infection group with 4485 units: immune group with 989 units,
 infection group with 801 units: immune group with 17 units}

In [101]:
f1 = fight(all_groups)
f1

[immune group with 905 units,
 infection group with 797 units,
 infection group with 4434 units]

In [102]:
target_selection(f1)

{immune group with 905 units: infection group with 797 units,
 infection group with 797 units: immune group with 905 units}

In [103]:
fight(f1)

[immune group with 761 units,
 infection group with 793 units,
 infection group with 4434 units]

In [105]:
while len(set([g.army for g in all_groups])) > 1:
    all_groups = fight(all_groups)
    print(all_groups)
sum([g.num_units for g in all_groups])

[immune group with 905 units, infection group with 797 units, infection group with 4434 units]
[immune group with 761 units, infection group with 793 units, infection group with 4434 units]
[immune group with 618 units, infection group with 789 units, infection group with 4434 units]
[immune group with 475 units, infection group with 786 units, infection group with 4434 units]
[immune group with 333 units, infection group with 784 units, infection group with 4434 units]
[immune group with 191 units, infection group with 783 units, infection group with 4434 units]
[immune group with 49 units, infection group with 782 units, infection group with 4434 units]
[infection group with 782 units, infection group with 4434 units]


5216

In [65]:
def attacker_priority(attacker):
    "For use in sorting attackers in target selection phase"
    return (attacker.effective_power, attacker.initiative)

In [66]:
def target_prioritisation(attacker):
    def target_priority(defender):
        "For use in sorting possible targets in target selection phase"
        return (maximal_damage(attacker, defender), defender.effective_power, defender.initiative)
    return target_priority

In [67]:
def maximal_damage(attacker, defender):
    if attacker.attack_type in defender.immunities:
        return 0
    elif attacker.attack_type in defender.weaknesses:
        return attacker.effective_power * 2
    else:
        return attacker.effective_power

In [177]:
test = "89 units each with 11269 hit points (weak to fire, radiation) with an attack that does 1018 slashing damage at initiative 7"

In [256]:
def parse_group_description(description):
    num_units, unit_hp, unit_power, initiative = Integers(description)
    attack_type = re.match(r".* (?P<attack_type>\w+) damage", description).group('attack_type')
    weaknesses = re.match(r"(.*weak to (?P<weaknesses>\w+(?:, \w+)*).*)?", description).group('weaknesses')
    weaknesses = weaknesses.split(', ') if weaknesses else None
    immunities = re.match(r"(.*immune to (?P<immunities>\w+(?:, \w+)*).*)?", description).group('immunities')
    immunities = immunities.split(', ') if immunities else None
    return {'num_units': num_units, 'unit_hp': unit_hp, 'unit_power': unit_power, 'initiative': initiative,
            'attack_type': attack_type, 'weaknesses': weaknesses, 'immunities': immunities}

In [257]:
parse_group_description(test)

{'attack_type': 'slashing',
 'immunities': None,
 'initiative': 7,
 'num_units': 89,
 'unit_hp': 11269,
 'unit_power': 1018,
 'weaknesses': ['fire', 'radiation']}

In [272]:
def create_groups():
    immune, infection = read_input(24).split('\n\n')
    start_groups = []
    for group_description in immune.splitlines()[1:]:
        attributes = parse_group_description(group_description)
        start_groups.append(Group(army='immune', **attributes))
    for group_description in infection.splitlines()[1:]:
        attributes = parse_group_description(group_description)
        start_groups.append(Group(army='infection', **attributes))
    return start_groups

In [309]:
def battle(all_groups):
    while len(set([g.army for g in all_groups])) > 1:
        all_groups = fight(all_groups)
    return all_groups

In [281]:
start_groups = create_groups()
all_groups = battle(start_groups)
sum([g.num_units for g in all_groups])

22996

In [282]:
all_groups

[infection group with 3186 units,
 infection group with 1252 units,
 infection group with 2241 units,
 infection group with 2590 units,
 infection group with 1650 units,
 infection group with 7766 units,
 infection group with 1790 units,
 infection group with 264 units,
 infection group with 2257 units]

In [311]:
increments = [100, 7, 1]
boost = 50

for increment in increments:
    while True:
        boost += increment
        print(boost)
        all_groups = create_groups()
        immune_army = [g for g in all_groups if g.army == 'immune']
        for g in immune_army:
            g.unit_power += boost        
        all_groups = battle(all_groups)
        if all_groups[0].army == 'immune':
            break
    if increment > 1:
        boost -= increment #we went too far
sum([g.num_units for g in all_groups])

150
57
51


4327

# Day 17

In [1]:
test_input = """x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""

In [2]:
def create_ground_slice(description):
    ground_slice = np.full((2000, 2000), '.')
    for line in description.split('\n'):
        ground_slice[parse_line(line)] = '#'
    return ground_slice

In [7]:
def parse_line(line):
    fixed, start, end = Integers(line)
    return (fixed, slice(start, end + 1)) if line[0] == 'y' else (slice(start, end + 1), fixed)

assert(parse_line('x=495, y=2..7') == (slice(2, 8), 495))
assert(parse_line('y=13, x=498..504') == (13, slice(498, 505)))

In [16]:
create_ground_slice(test_input)[:20, 495:507]

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['#', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['#', '.', '.', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['#', '.', '.', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['#', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['#', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['#', '#', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '#', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '#', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '#', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '#', '#', '#', '#', '#', '#', '#', '.', '.'],
       ['.', '.', '.', '.', '.', '

In [4]:
def below(x, y):
    return (x + 1, y)

def above(x, y):
    return (x - 1, y)

def left(x, y):
    return (x, y - 1)

def right(x, y):
    return (x, y + 1)

In [38]:
def explore_below(start, ground_slice):
    current_loc = start
    loc_type = ground_slice[current_loc]
    while loc_type in '.|':
        current_loc = below(*current_loc)
        try:
            loc_type = ground_slice[current_loc]
        except IndexError:
            break
    return above(*current_loc)

def explore_sideways(direction, start, ground_slice):
    current_loc = start
    while ground_slice[current_loc] in '.|' and ground_slice[below(*current_loc)] in '~#':
        current_loc = direction(*current_loc)
    if ground_slice[current_loc] in '~#':
        return (current_loc, 'blocked')
    else:
        return (current_loc, 'overflow')

In [83]:
def pour(source, ground_slice, poured=[]):
    poured.append(source)
    bottom = explore_below(source, ground_slice)
    ground_slice[source[0]:bottom[0], source[1]] = '|'
    if below(*bottom)[0] == ground_slice.shape[0]:
        return
    while True:
        leftmost, l_status = explore_sideways(left, bottom, ground_slice)
        rightmost, r_status = explore_sideways(right, bottom, ground_slice)
        spread = (leftmost[0], slice(right(*leftmost)[1], rightmost[1]))
        if l_status == 'blocked' and r_status == 'blocked':
            ground_slice[spread] = '~'
            bottom = above(*bottom)
        else:
            ground_slice[spread] = '|'
            break
    if l_status == 'overflow' and leftmost not in poured:
        pour(leftmost, ground_slice)
    if r_status == 'overflow' and rightmost not in poured:
        pour(rightmost, ground_slice)

In [65]:
test_slice = create_ground_slice(test_input)
pour((0, 500), test_slice)
test_slice[:20, 495:507]

array([['.', '.', '.', '.', '.', '|', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '|', '.', '.', '.', '.', '.', '#'],
       ['#', '.', '.', '#', '|', '|', '|', '|', '.', '.', '.', '#'],
       ['#', '.', '.', '#', '~', '~', '#', '|', '.', '.', '.', '.'],
       ['#', '.', '.', '#', '~', '~', '#', '|', '.', '.', '.', '.'],
       ['#', '~', '~', '~', '~', '~', '#', '|', '.', '.', '.', '.'],
       ['#', '~', '~', '~', '~', '~', '#', '|', '.', '.', '.', '.'],
       ['#', '#', '#', '#', '#', '#', '#', '|', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '|', '.', '.', '.', '.'],
       ['.', '.', '|', '|', '|', '|', '|', '|', '|', '|', '|', '.'],
       ['.', '.', '|', '#', '~', '~', '~', '~', '~', '#', '|', '.'],
       ['.', '.', '|', '#', '~', '~', '~', '~', '~', '#', '|', '.'],
       ['.', '.', '|', '#', '~', '~', '~', '~', '~', '#', '|', '.'],
       ['.', '.', '|', '#', '#', '#', '#', '#', '#', '#', '|', '.'],
       ['.', '.', '|', '.', '.', '

In [89]:
def water_sum(array):
    clay_rows = [i for i in range(len(array)) if '#' in array[i]]
    valid_slice = array[min(clay_rows):max(clay_rows) + 1]
    return (valid_slice == '~').sum() + (valid_slice == '|').sum()

In [91]:
def water_sum2(array):
    clay_rows = [i for i in range(len(array)) if '#' in array[i]]
    valid_slice = array[min(clay_rows):max(clay_rows) + 1]
    return (valid_slice == '~').sum()

In [92]:
water_sum(test_slice)

57

In [93]:
water_sum2(test_slice)

29

In [84]:
ground_slice = create_ground_slice(read_input(17))
pour((0, 500), ground_slice)

In [90]:
water_sum(ground_slice)

34541

In [94]:
water_sum2(ground_slice)

28000

# Day 20

In [5]:
direction_map = {'N': above, 'S': below, 'E': right, 'W': left}

In [72]:
def parse_regex(regex, start_node):
    trackbacks = [] # list of points to return to in case of branches (LIFO)
    branch_ends = [] # list of sets of points to add to current_nodes
    current_nodes = [start_node]
    for char in regex:
        if char == '$':
            pass
        elif char == '^':
            pass
        elif char == '(':
            trackbacks.append(tuple(current_nodes))
            branch_ends.append([])
        elif char == ')':
            branch_ends[-1].append(tuple(current_nodes))
            current_nodes = list(set(flatten(branch_ends.pop())))
            trackbacks.pop()
        elif char == '|':
            branch_ends[-1].append(tuple(current_nodes))
            current_nodes = list(trackbacks[-1])
        else:
            direction = direction_map[char]
            for i in range(len(current_nodes)):
                transition_node = direction(*current_nodes[i])
                destination_node = direction(*transition_node)
                yield (current_nodes[i], transition_node, destination_node)
                current_nodes[i] = destination_node

In [73]:
for i in parse_regex('^ENWWW(NEEE|SSE(EE|N))$', (5, 5)):
    pass

In [74]:
def flatten(sequence):
    return [x for item in sequence for x in item]

In [118]:
def create_map(regex, size=21):
    facility_map = np.full((size, size), '?')
    start_node = (size//2, size//2)
    facility_map[start_node] = '.'
    for start, trans, destination in parse_regex(regex, start_node):
        facility_map[trans] = '/'
        facility_map[destination] = '.'
    facility_map[facility_map == '?'] = '#'
    return facility_map, start_node

In [76]:
def print_map(m):
    for row in m:
        print(''.join(row))

In [119]:
test_regex = '^ENWWW(NEEE|SSE(EE|N))$'
m, start = create_map(test_regex, size=21)
print_map(m)

#####################
#####################
#####################
#####################
#####################
#####################
######./././.########
######/##############
######./././.########
######/#####/########
######.#.#./.########
######/#/############
######./././.########
#####################
#####################
#####################
#####################
#####################
#####################
#####################
#####################


In [121]:
max(get_room_distances(m, start).values())

10

In [123]:
test_regex = '^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$'
m, start = create_map(test_regex, size=21)
max(get_room_distances(m, start).values())

18

In [124]:
test_regex = '^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$'
m, start = create_map(test_regex, size=21)
max(get_room_distances(m, start).values())

23

In [125]:
test_regex = '^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$'
m, start = create_map(test_regex, size=21)
max(get_room_distances(m, start).values())

31

In [126]:
m, start = create_map(read_input(20), size=301)
max(get_room_distances(m, start).values())

3958

In [127]:
sum([x >= 1000 for x in get_room_distances(m, start).values()])

8566

In [105]:
import heapq

In [117]:
def get_room_distances(facility_map, start):
    room_heap = [] # we will maintain a heap of (passed_doors, room) tuples
    heapq.heappush(room_heap, (0, start))
    room_distances = {start: 0}
    while room_heap:
        doors, room = heapq.heappop(room_heap)
        neibouring_rooms = [n for n in get_neighbouring_rooms(room, facility_map) if n not in room_distances]
        for n in neibouring_rooms:
            heapq.heappush(room_heap, (doors + 1, n))
            room_distances[n] = doors + 1
    return room_distances   

In [108]:
def get_neighbouring_rooms(room, facility_map):
    neighbouring_rooms = []
    for direction in direction_map.values():
        if facility_map[direction(*room)] == '/':
            neighbouring_rooms.append(direction(*direction(*room)))
    return neighbouring_rooms

# Day 23

In [128]:
def manhattan_distance(a, b):
    if len(a) != len(b):
        raise ValueError('a and b must be the same length')
    return sum([abs(b[i] - a[i]) for i in range(len(a))])
    
assert(manhattan_distance((0,0), (3,4)) == 7)
assert(manhattan_distance((0,-1,0), (1,1,2)) == 5)

In [129]:
nanobot_data = mapt(Integers, read_input(23).splitlines())
nanobot_data[:5]

((-33594389, 69103993, 46909087, 93546878),
 (67619021, 22999634, 40676275, 81086929),
 (111047329, 44087494, 53754703, 96875177),
 (49661119, 55848282, 9865303, 91139231),
 (40162628, 85216421, 62921753, 68894464))

In [130]:
radiuses = [d[3] for d in nanobot_data]
positions = [d[:3] for d in nanobot_data]

In [131]:
strongest_radius = max(radiuses)
strongest_index = radiuses.index(strongest_radius)
strongest_position = positions[strongest_index]

In [132]:
sum([1 for p in positions if manhattan_distance(p, strongest_position) <= strongest_radius])

396

### Part 2

In [133]:
def bots_in_range(point, bot_locations, bot_radiuses):
    return sum([1 for i in range(len(bot_locations)) if manhattan_distance(bot_locations[i], point) <= bot_radiuses[i]])

The idea is to divide space up into cubes of side length $2^{n}$. First we start with a cube big enough such that it is in range of all bots, and put this cube to the top of a priority queue, in the form of a (bots_in_range, dist_to_origin, cube) tuple. Then the algorithm is as follows:

- pop the top of the priority queue
- if the cube has side length = 1, this is the best
- divide into 8 subcubes (of half the side length) and add these to the queue

This works because when a cube of side length 1 gets to the top, nothing behind can beat it since everything else will either have fewer potential bots in range of be further from the origin.

In [159]:
def bot_extremities(bot):
    x, y, z, radius = bot
    return ((x+radius, y, z), (x-radius, y, z), (x, y+radius, z), (x, y-radius, z), (x, y, z+radius), (x, y, z-radius))

In [201]:
def cube_extremities(cube):
    x, y, z, side_length = cube
    step = side_length - 1
    return ((x, y, z), (x+step, y, z), (x, y+step, z), (x, y, z+step),
            (x+step, y+step, z), (x+step, y, z+step), (x, y+step, z+step), 
            (x+step, y+step, z+step))

In [153]:
def in_cube(point, cube):
    *bottom_corner, side_length = cube
    step = side_length - 1
    return all([bottom_corner[i] <= point[i] <= bottom_corner[i] + step for i in range(len(point))])

assert(in_cube((1,1,1), (0,0,0,2)))
assert(not in_cube((1,1,1), (0,0,0,1)))

In [192]:
def has_intersection(bot, cube):
    "sketchy"
    cube_in_bot = any([manhattan_distance(ce, bot[:3]) <= bot[3] for ce in cube_extremities(cube)])
    bot_in_cube = any([in_cube(be, cube) for be in bot_extremities(bot)])
    return cube_in_bot or bot_in_cube

In [155]:
def split(cube):
    x, y, z, side_length = cube
    if side_length % 2 != 0:
        raise ValueError("cube is not of even length - cannot split")
    step = side_length // 2
    return ((x, y, z, step), (x+step, y, z, step), (x, y+step, z, step), (x, y, z+step, step),
            (x+step, y+step, z, step), (x+step, y, z+step, step), (x, y+step, z+step, step), 
            (x+step, y+step, z+step, step))

In [211]:
def min_cube_dist(cube):
    "sketchy"
    return min([manhattan_distance(ce, (0, 0, 0)) for ce in cube_extremities(cube)])

In [215]:
def get_initial_cube(bots):
    i=1
    while True:
        side_length = 2**i
        bottom_corner = (-side_length//2, -side_length//2, -side_length//2)
        cube = (*bottom_corner, side_length)
        if sum([has_intersection(bot, cube) for bot in bots]) >= len(bots):
            return cube
        i += 1

In [209]:
initial_cube(nanobot_data)

(-268435456, -268435456, -268435456, 536870912)

In [225]:
def get_best_point(bots):
    initial_cube = get_initial_cube(bots)
    cube_heap = [] # we will maintain a heap of (-bots_in_range, dist_to_origin, cube) tuples
    origin = (0, 0, 0)
    initial_element = (-len(bots), manhattan_distance(initial_cube[:3], origin), initial_cube)
    heapq.heappush(cube_heap, initial_element)
    while cube_heap:
        *_, cube = heapq.heappop(cube_heap)
        if cube[3] == 1:
            return cube
        sub_cubes = split(cube)
        for c in sub_cubes:
            bots_in_range = sum([has_intersection(bot, c) for bot in bots])
            distance = min_cube_dist(c)
            print(bots_in_range)
            heapq.heappush(cube_heap, (-bots_in_range, distance, c))

In [226]:
get_best_point(nanobot_data)

45
78
171
165
773
693
755
993
987
311
325
200
7
4
1
0
975
748
900
871
673
563
713
526
594
737
753
773
849
844
974
928
796
949
788
907
915
974
752
928
974
926
974
969
953
838
972
905
724
833
849
369
944
932
974
974
348
841
813
359
950
470
341
974
929
944
902
945
921
973
825
900
945
935
974
974
941
823
974
938
348
974
845
365
952
472
345
869
223
826
945
323
827
324
354
825
818
940
918
945
920
942
949
942
348
843
850
370
850
370
348
974
974
945
974
974
935
820
938
938
843
945
945
945
942
827
974
945
348
370
348
370
850
370
243
472
354
827
974
354
820
318
318
820
843
945
974
472
945
827
974
974
949
942
949
974
913
935
913
935
821
843
850
872
920
945
952
974
354
354
472
354
820
318
213
318
472
974
974
472
974
354
367
940
952
971
949
974
913
938
811
935
348
974
847
367
952
472
345
869
223
827
945
325
827
325
354
827
818
942
920
945
920
942
949
942
348
370
348
370
850
370
243
472
847
940
813
835
808
833
811
833
974
945
974
974
937
822
940
938
843
945
974
472
945
827
974
974
827
793
940
822
78

(21131263, 40824380, 57450697, 1)

In [227]:
sum([has_intersection(bot, (21131263, 40824380, 57450697, 1)) for bot in nanobot_data])

974

In [228]:
manhattan_distance((21131263, 40824380, 57450697), (0, 0, 0))

119406340